In [ ]:
!pip install ultralytics
!pip install opencv-python-headless

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

### 원본데이터 복사

In [ ]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
import shutil

def copy_files(src_dir, dst_dir, file_ext):
    os.makedirs(dst_dir, exist_ok=True)
    for root, _, files in os.walk(src_dir):
        for file in files:
            if file.endswith(file_ext):
                src_path = os.path.join(root, file)
                dst_path = os.path.join(dst_dir, file)
                shutil.copy2(src_path, dst_path)

# 훈련 데이터 복사
copy_files('/gdrive/MyDrive/Final project/skindata/Training/01.원천데이터', '/content/yolo_dataset/train/images', '.jpg')
# copy_files('/gdrive/MyDrive/Final project/skindata/Training/02.라벨링데이터', '/content/yolo_dataset/train/labels', '.json')

# 검증 데이터 복사
copy_files('/gdrive/MyDrive/Final project/skindata/Validation/01.원천데이터', '/content/yolo_dataset/val/images', '.jpg')
# copy_files('/gdrive/MyDrive/Final project/skindata/Validation/02.라벨링데이터', '/content/yolo_dataset/val/labels', '.json')

In [ ]:
!ls -R /content/yolo_dataset

In [ ]:
# 런타임 연결 끊김 방지
from IPython.display import display, Javascript
from google.colab import output

display(Javascript('''
function ClickConnect(){
    console.log("Clicking connect button");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ClickConnect, 60000)

function KeepClicking(){
    console.log("Clicking");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(KeepClicking, 60000)
'''))

output.enable_custom_widget_manager()

In [ ]:
from IPython.display import display, Javascript
from google.colab import output

display(Javascript('''
function ClickConnect(){
    console.log("Clicking connect button");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}

// 매 5분마다 연결 버튼 클릭
setInterval(ClickConnect, 300000)

// 페이지가 유휴 상태일 때 마우스 움직임 시뮬레이션
function simulateMouseMove() {
    var event = new MouseEvent('mousemove', {
        'view': window,
        'bubbles': true,
        'cancelable': true
    });
    document.dispatchEvent(event);
}

// 매 2분마다 마우스 움직임 시뮬레이션
setInterval(simulateMouseMove, 120000)
'''))

# 출력 창 지우기 방지
output.enable_custom_widget_manager()


In [ ]:
# 이미 생겨버린 /content/dataset.yaml 제거하기 -> yolo 다시 학습해야할 경우
# import os

# if os.path.exists('/content/dataset.yaml'):
#     os.remove('/content/dataset.yaml')
#     print("Existing dataset.yaml removed")

### 도전 중

In [ ]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont

# 1. 데이터 준비
def prepare_yolo_data(image_base_dir, label_base_dir, output_dir):
    # 출력 디렉토리 생성 (이미 존재하면 무시)
    os.makedirs(output_dir, exist_ok=True)

    # 장치 유형별로 반복 ('1. 디지털카메라', '2. 스마트패드', '3. 스마트폰')
    for device in ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰']:
        image_device_dir = os.path.join(image_base_dir, device)
        label_device_dir = os.path.join(label_base_dir, device)

        for subfolder in os.listdir(image_device_dir):
            image_subfolder_path = os.path.join(image_device_dir, subfolder)
            label_subfolder_path = os.path.join(label_device_dir, subfolder)

            for img_file in os.listdir(image_subfolder_path):
                if img_file.endswith('.jpg'):
                    # 전체 이미지 파일 경로 생성
                    img_path = os.path.join(image_subfolder_path, img_file)
                    # OpenCV를 사용하여 이미지 읽기
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Warning: Could not read image {img_path}")
                        continue
                    # 이미지의 높이와 너비 가져오기
                    h, w = img.shape[:2]

                    # 파일 이름에서 확장자 제거
                    base_name = os.path.splitext(img_file)[0]
                    # YOLO 형식의 라벨 파일 생성
                    with open(os.path.join(output_dir, f"{base_name}.txt"), 'w') as f:
                        # 9개의 얼굴 부위에 대해 반복
                        for i in range(9):
                            # (예: 0001_03_F_00.json)
                            json_file = f"{base_name}_{i:02d}.json"
                            # JSON 파일 경로 생성
                            json_path = os.path.join(label_subfolder_path, json_file)

                            if not os.path.exists(json_path):
                                print(f"JSON file not found: {json_path}")
                                continue

                            try:
                                # JSON 파일 읽기
                                with open(json_path, 'r', encoding='utf-8') as jf:
                                    data = json.load(jf)

                                # 바운딩 박스 정보 추출
                                bbox = data['images']['bbox']
                                # YOLO 형식으로 좌표 변환 (중심 x, 중심 y, 너비, 높이)
                                x_center = (bbox[0] + bbox[2]) / (2 * w)
                                y_center = (bbox[1] + bbox[3]) / (2 * h)
                                width = (bbox[2] - bbox[0]) / w
                                height = (bbox[3] - bbox[1]) / h
                                # YOLO 형식으로 라벨 쓰기
                                f.write(f"{i} {x_center} {y_center} {width} {height}\n")
                                print(f"Processed {json_file}")
                            except Exception as e:
                                print(f"Error processing {json_file}: {str(e)}")

# 2. YOLO 모델 학습
def train_yolo_model(data_yaml_path):
    model = YOLO('yolov8n.yaml')
    model.train(data=data_yaml_path, epochs=50, imgsz=640)
    return model

# 메인 실행 코드
if __name__ == "__main__":
    # YAML 파일 생성
    yaml_content = f"""
path: /gdrive/MyDrive/Final project/skindata
train: Training/01.원천데이터
val: Validation/01.원천데이터

nc: 9
names: ['forehead', 'glabella', 'left_eye', 'right_eye', 'left_cheek', 'right_cheek', 'nose', 'mouth', 'chin']
"""
    with open('/gdrive/MyDrive/Final project/dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # 1. 데이터 준비
    prepare_yolo_data('/gdrive/MyDrive/Final project/skindata/Training/01.원천데이터',
                      '/gdrive/MyDrive/Final project/skindata/Training/02.라벨링데이터',
                      '/gdrive/MyDrive/Final project/yolo_dataset/train/labels')
    prepare_yolo_data('/gdrive/MyDrive/Final project/skindata/Validation/01.원천데이터',
                      '/gdrive/MyDrive/Final project/skindata/Validation/02.라벨링데이터',
                      '/gdrive/MyDrive/Final project/yolo_dataset/val/labels')

    # 2. YOLO 모델 학습
    model = train_yolo_model('/gdrive/MyDrive/Final project/dataset.yaml')

In [ ]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ResNet-50 기반 피부 진단 모델 정의
class SkinDiagnosisModel(nn.Module):
    def __init__(self, num_classes):
        super(SkinDiagnosisModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

# 커스텀 데이터셋 클래스 정의
class FacePartDataset(Dataset):
    def __init__(self, image_dir, label_dir, facepart, transform=None):
        self.image_dir = image_dir  # 이미지 디렉토리 경로
        self.label_dir = label_dir  # 라벨 디렉토리 경로
        self.facepart = facepart  # 얼굴 부위 인덱스
        self.transform = transform  # 이미지 변환 함수
        self.data = self.load_data()  # 데이터 로드


    def load_data(self):
        data = []
        for img_file in os.listdir(self.image_dir):
            if img_file.endswith('.jpg'):
                base_name = os.path.splitext(img_file)[0]
                json_file = f"{base_name}_{self.facepart:02d}.json"
                json_path = os.path.join(self.label_dir, json_file)

                if os.path.exists(json_path):
                    with open(json_path, 'r') as f:
                        label_data = json.load(f)

                    features = self.extract_features(label_data)
                    data.append((os.path.join(self.image_dir, img_file), features))
        return data

    def extract_features(self, label_data):
        features = {}
        if self.facepart == 0:
            features['skin_type'] = label_data['info']['skin_type']
            features['sensitive'] = label_data['info']['sensitive']
        else:
            if 'annotations' in label_data:
                features.update(label_data['annotations'])
            if 'equipment' in label_data and label_data['equipment'] is not None:
                features.update(label_data['equipment'])
        return features

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, features = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        feature_vector = torch.tensor([float(v) if isinstance(v, (int, float)) else 0 for v in features.values()])
        return image, feature_vector

# 각 얼굴 부위별 ResNet-50 모델 학습 함수
def train_skin_diagnosis_model(facepart, train_data, val_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU 사용 가능 여부 확인
    num_classes = len(train_data.dataset[0][1])  # 출력 클래스 수를 입력 특성 수와 동일하게 설정
    model = SkinDiagnosisModel(num_classes).to(device) # 모델 초기화
    criterion = nn.MSELoss() # 손실 함수 정의
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 정의

    num_epochs = 10

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_data:
            inputs, labels = inputs.to(device), labels.to(device) # 데이터를 GPU로 이동
            optimizer.zero_grad() # 그래디언트 초기화
            outputs = model(inputs)
            loss = criterion(outputs, labels) # 손실 계산
            loss.backward()
            optimizer.step() # 가중치 업데이트

        # 검증
        model.eval() # 평가 모드로 전환
        val_loss = 0
        with torch.no_grad(): # 그래디언트 계산 비활성화
            for inputs, labels in val_data:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()

        print(f'Facepart: {facepart}, Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss/len(val_data):.4f}')

    return model

# 수정된 피부 진단 서비스 함수
def skin_diagnosis_service(image_path, yolo_model, skin_models):
    image = Image.open(image_path)
    image_tensor = transforms.ToTensor()(image).unsqueeze(0)

    yolo_results = yolo_model(image)  # YOLO 모델로 얼굴 부위 탐지

    diagnoses = {}
    for result in yolo_results:
        for i, box in enumerate(result.boxes.xyxy):
            part_name = result.names[int(result.boxes.cls[i])]  # 탐지된 얼굴 부위 이름

            x1, y1, x2, y2 = box
            part_tensor = image_tensor[:, :, int(y1):int(y2), int(x1):int(x2)]  # 얼굴 부위 잘라내기

            skin_model = skin_models[part_name]  # 해당 부위의 ResNet-50 모델 선택
            skin_model.eval()
            with torch.no_grad():
                diagnosis = skin_model(part_tensor)  # ResNet-50 모델로 피부 상태 진단

            diagnoses[part_name] = diagnosis.squeeze().tolist()

    return diagnoses

# 수정된 모델 평가 함수
def evaluate_model(test_image_path, yolo_model, skin_models, label_base_path):
    diagnoses = skin_diagnosis_service(test_image_path, yolo_model, skin_models)

    base_name = os.path.splitext(os.path.basename(test_image_path))[0]
    true_labels = {}
    for i in range(9):
        json_file = f"{base_name}_{i:02d}.json"
        json_path = os.path.join(label_base_path, json_file)
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                data = json.load(f)
                if i == 0:
                    true_labels[i] = [data['info']['skin_type'], data['info']['sensitive']]
                else:
                    features = {}
                    if 'annotations' in data:
                        features.update(data['annotations'])
                    if 'equipment' in data and data['equipment'] is not None:
                        features.update(data['equipment'])
                    true_labels[i] = [float(v) if isinstance(v, (int, float)) else 0 for v in features.values()]

    total_mse = 0
    for i, part in enumerate(diagnoses):
        if i in true_labels:
            mse = np.mean((np.array(diagnoses[part]) - np.array(true_labels[i]))**2)
            total_mse += mse
            print(f"Part {i} MSE: {mse:.4f}")

    avg_mse = total_mse / len(true_labels)
    print(f"평균 MSE: {avg_mse:.4f}")

    return avg_mse

# 메인 실행 코드
if __name__ == "__main__":
    # YOLO 모델 로드
    yolo_model = YOLO('yolov8n.pt')  # 또는 학습된 모델 경로

    # 각 얼굴 부위별 ResNet-50 모델 학습
    faceparts = ['forehead', 'glabella', 'left_eye', 'right_eye', 'left_cheek', 'right_cheek', 'nose', 'mouth', 'chin']
    skin_models = {}

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    for i, facepart in enumerate(faceparts):
        train_dataset = FacePartDataset(
            '/gdrive/MyDrive/Final project/skindata/Training/01.원천데이터',
            '/gdrive/MyDrive/Final project/skindata/Training/02.라벨링데이터',
            i,
            transform=transform
        )
        val_dataset = FacePartDataset(
            '/gdrive/MyDrive/Final project/skindata/Validation/01.원천데이터',
            '/gdrive/MyDrive/Final project/skindata/Validation/02.라벨링데이터',
            i,
            transform=transform
        )

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        skin_models[facepart] = train_skin_diagnosis_model(facepart, train_loader, val_loader)

    # 학습된 모델 저장
    torch.save(skin_models, '/gdrive/MyDrive/Final project/skin_diagnosis_models.pth')

    # 테스트 및 평가
    test_image_path = '/gdrive/MyDrive/Final project/skindata/test/01.원천데이터/1091_03_F.jpg'
    label_base_path = '/gdrive/MyDrive/Final project/skindata/test/02.라벨링데이터'
    avg_mse = evaluate_model(test_image_path, yolo_model, skin_models, label_base_path)